In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sc
from cvxopt import solvers,matrix 

In [2]:
import os
os.listdir()

['.DS_Store',
 'state_of_the_art_hard_margin.ipynb',
 '.ipynb_checkpoints',
 'spamdata.mat',
 '771A_lec8_slides.pdf',
 'ACFrOgBA8JRQCnWZNF2y9MGVmmmF98EOE4PqZKLg9TZ1A0CvsQWvcuXC-3CL8jVSyp9rk6Fu0E3ujkwkgt8up0YOSIBYZFstEDQuFvTitC5bOK9Ih14Z5qbMnuOS3og=.pdf']

In [3]:
data = sc.loadmat('spamdata.mat')

In [4]:
list = ['xtrain', 'ytrain', 'xtest', 'ytest']

In [5]:
xtest, xtrain, ytest, ytrain = data['xtrain'], data['xtest'], data['ytrain'], data['ytest']

In [6]:
xtrain.shape,ytrain.shape,xtest.shape, ytest.shape

((2000, 57), (2000, 1), (500, 57), (500, 1))

In [7]:
xtrain.shape

(2000, 57)

Our dual problem is expressed as:
$\max_{\alpha} \sum_i^m \alpha_i - \frac{1}{2} \sum_{i,j}^m y^{(i)}y^{(j)} \alpha_i \alpha_j <x^{(i)} x^{(j)}>$

Let H be a matrix such that $H_{i,j} = y^{(i)}y^{(j)} <x^{(i)} x^{(j)}>$ , then the optimization becomes:

$\begin{aligned}
    & \max_{\alpha} \sum_i^m \alpha_i  - \frac{1}{2}  \alpha^T \mathbf{H}  \alpha
    \\
     s.t. & \ \alpha_i \geq 0 
    \\
    &  \ \sum_i^m \alpha_i y^{(i)} = 0  
\end{aligned}$

Equation to be solved using Solver
$\begin{aligned}
    & \min \frac{1}{2} x^TPx + q^Tx
    \\
     s.t. \ & \ Gx \leq h 
    \\
    & \ Ax = b
\end{aligned}$

In [8]:
m,n = xtrain.shape
ytrain = ytrain.reshape(-1,1) * 1.
X_dash = ytrain * xtrain
H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format
P = matrix(H)
q = matrix(-np.ones((m, 1)))
G = matrix(-np.eye(m))
h = matrix(np.zeros(m))
A = matrix(ytrain.reshape(1, -1))
b = matrix(np.zeros(1))

sol = solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -8.0766e+02 -3.8120e+03  3e+04  1e+02  5e+00
 1: -2.1291e+03 -7.5066e+03  2e+04  9e+01  4e+00
 2: -4.0466e+03 -1.1583e+04  2e+04  7e+01  3e+00
 3: -6.8509e+03 -1.7138e+04  2e+04  6e+01  2e+00
 4: -1.2460e+04 -2.4511e+04  2e+04  5e+01  2e+00
 5: -2.4424e+04 -3.8347e+04  2e+04  4e+01  2e+00
 6: -4.1428e+04 -5.9380e+04  2e+04  4e+01  1e+00
 7: -4.9789e+04 -7.0306e+04  2e+04  4e+01  1e+00
 8: -1.3788e+05 -1.6557e+05  3e+04  3e+01  1e+00
 9: -3.2653e+05 -3.7867e+05  5e+04  3e+01  1e+00
10: -9.5556e+05 -1.0680e+06  1e+05  3e+01  1e+00
11: -4.1485e+06 -4.4639e+06  3e+05  3e+01  1e+00
12: -1.5262e+07 -1.6201e+07  9e+05  3e+01  1e+00
13: -1.3535e+08 -1.3987e+08  5e+06  3e+01  1e+00
14: -5.1586e+08 -5.3130e+08  2e+07  3e+01  1e+00
15: -5.1654e+08 -5.3199e+08  2e+07  3e+01  1e+00
16: -5.2133e+08 -5.3677e+08  2e+07  3e+01  1e+00
Terminated (singular KKT matrix).


In [9]:
alphas.shape

(2000, 1)

In [10]:
w = ((ytrain * alphas).T @ xtrain).reshape(-1,1)

S = (alphas > 1e-4).flatten()

b = ytrain[S] - np.dot(xtrain[S], w)

print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

Alphas =  [342162.94793939 134434.23098234  32532.43827815 ...   1686.26713287
 557538.52491428 191765.80204683]
w =  [-3.68862506e-03 -2.72541679e-03  2.39631347e-03  2.23090523e-03
  3.43648903e-03  1.98951690e-03  4.87624947e-03 -6.06043264e-04
 -1.62086869e-03  1.51494518e-03  9.25127836e-03  1.09165907e-04
  1.24016777e-04 -4.25396863e-03  6.92067295e-03  3.50269070e-03
  9.64724831e-03  4.00851294e-03 -5.48943877e-04  1.05295721e-02
  1.19778700e-03  6.57060079e-03  3.28283380e-02  1.94522631e-02
 -6.10632449e-03 -7.54110515e-04 -4.00717510e-03 -4.28592786e-04
 -6.23642141e-03 -2.59834807e-03 -3.19496728e-04  6.80176038e-02
  2.09021568e-03 -5.11894582e-02 -1.92201440e-02  7.44693447e-03
 -1.68622471e-04 -6.72999595e-04 -1.20855889e-02 -4.80626733e-03
 -2.56961748e-03 -1.19084315e-02  2.64571793e-03 -2.86572957e-02
 -5.12247439e-03 -7.39713944e-03 -3.56334445e-02 -4.39670343e-02
 -3.26519881e-02  2.65846774e-03 -2.03737721e-02  1.30632520e-03
  3.64748025e-02  8.61564832e-03 -7.7

In [18]:
error = np.abs((xtest @ w  > 0) * 1 - (ytest > 0)).mean()

In [19]:
accuracy = 1 - error

In [20]:
print("Accuracy and  error on Test set is {} and {}".format(accuracy*100, error*100))

Accuracy and  error on Test set is 84.39999999999999 and 15.6


In [21]:
np.sum(alphas * ytrain)

-2.903863787651062e-06

In [22]:
b

array([[ 0.99312707],
       [ 0.97277672],
       [-0.94998345],
       ...,
       [-0.96353402],
       [ 0.98864695],
       [-1.0034413 ]])

In [25]:
error1 = np.abs((xtrain @ w  > 0) * 1 - (ytrain > 0)).mean()

In [26]:
accuracy1 = 1 - error1

In [28]:
print("Accuracy and  error on Training set is {} and {}".format(accuracy1*100, error1*100))

Accuracy and  error on Training set is 86.8 and 13.200000000000001
